In [ ]:
%matplotlib inline
%reload_ext autoreload 
%autoreload 2
import numpy as np

import matplotlib.pyplot as plt
import wavio as wav
from strauss.sonification import Sonification
from strauss.sources import Events
from strauss import channels
from strauss.score import Score
import numpy as np
from strauss.generator import Sampler
import IPython.display as ipd
import os
import pandas as pd
from datetime import datetime as dt
from matplotlib import patheffects

from scipy.io.wavfile import write
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal
from pathlib import Path

from pycbc.psd.analytical import aLIGO140MpcT1800545
from pycbc.waveform import get_td_waveform

In [ ]:
# output sampling rate
SAMPRATE = 48000

# base total mass for calculation (degenerate with PLO, keep the same)
MBASE = 40

# Frames per second for video
FPS = 30

# Length of sequence in seconds :: 90s = 1m 30s
DURATION = 90

# relative overall pitch shift in semitones compared with fiducial setup:
PLO = 0

# seed randoms
np.random.seed(42)

## Read Events

In [ ]:
# detections downloaded from gwosc.com
dets = pd.read_csv('detections.csv')

# supplementary file costructed from chain (ML positions courtesy of Michael Williams)
pos = pd.read_csv('gw_parameters_ra_dec_dl.csv')

In [ ]:
tstamps = []

dfmt = "%y%m%d_%H%M%S"
for i in range(len(dets['id'])):
    date = dets['id'][i][2:].split('-')[0]
    if len(date.split('_')) < 2:
        date += "_000000"
    dtime = dt.strptime(date, dfmt)
    tstamps.append(dt.timestamp(dtime))

tstamps = np.array(tstamps)
dets['tstamp'] = tstamps


In [ ]:
dets['mrat'] = dets['mass_1_source'].values / dets['mass_2_source'].values
dets['mtot'] = (dets['mass_1_source'].values + dets['mass_2_source'].values)
dets.dropna(subset=['mrat','mtot','luminosity_distance'], inplace=True)

## Make Waveforms 

_Note: Because the event times are for the merger itself (which happens ~at end of waveform), and samples are quite long compared to sonficiation times, it makes sense to sonify time backwards with reversed samples, and reverse everything at the end! That's why we save the samples as reversed._

In [ ]:
# get keyboard notes for sampler loading...

cmaj = list('CDEFGAB')

notes = list(('1,'.join(cmaj)+'1').split(',')) + \
        list(('2,'.join(cmaj)+'2').split(',')) + \
        list(('3,'.join(cmaj)+'3').split(',')) + \
        list(('4,'.join(cmaj)+'4').split(',')) + \
        list(('5,'.join(cmaj)+'5').split(',')) + \
        list(('6,'.join(cmaj)+'6').split(','))

In [ ]:
# construct sample bank for different mass ratios

def waveform(mass_ratio = 1., mtotal = MBASE):
    hp, _ = get_td_waveform(approximant='IMRPhenomXAS',
                            mass1 = mtotal * mass_ratio / (1. + mass_ratio),
                            mass2 = mtotal / (1. + mass_ratio),
                            f_lower = 25,
                            delta_t = 1./SAMPRATE)
    return hp.numpy() / hp.max(), hp.max()



mrat_pcs = np.percentile(dets['mrat'].values, np.linspace(0,100, len(notes)+1))
mrat_pcs[-1] += 1e-5
mratbins = np.digitize(dets['mrat'].values, mrat_pcs,right=False)


# Iterate for audio sample subset and save GW sample pack...
sdir = './samples/gws'
Path(sdir).mkdir(parents=True, exist_ok=True)
for i in range(len(notes)):
    q = 0.5*(mrat_pcs[i] + mrat_pcs[i+1])
    wf, _ = waveform(q, MBASE)
    write(f"{sdir}/gws_{notes[i]}.wav", SAMPRATE, (wf[::-1]*(pow(2,31)-1)).astype(np.int32))

# Iterate through all events for intrinsic peak strain amplitudes... (NB could also use SNRs...)
intrinsic_amps = []
for i in range(len(dets['mtot'])):
        wf, h = waveform(dets['mrat'].values[i], dets['mtot'].values[i])
        intrinsic_amps.append(h)
intrinsic_amps = np.array(intrinsic_amps)
intrinsic_amps /= intrinsic_amps.max()

dets['mratbin'] = mratbins
dets['hrelnorm'] = np.array(intrinsic_amps)[mratbins-1]/dets['luminosity_distance'].values
dets['hrelnorm'] /= dets['hrelnorm'].max()


# random directions for now
dets['ra'] = 2 * np.pi * np.random.random(len(dets['mrat']))
dets['dec'] = np.arcsin(1 - 2 * np.random.random(len(dets['mrat']))) 
plt.scatter(dets['ra'], dets['dec'])

In [ ]:
# cross match positions to detections with all the one's we have, using random positions for those we don't

conforming_ids = []
for i in range(pos['id'].values.size):
    id = pos['id'][i]
    if int(id[2:4]) <= 17:
        id = id.split('_')[0]
    conforming_ids.append(id)
print(conforming_ids)
pos['commonName'] = conforming_ids
dets = pd.merge(dets,pos[['ra', 'dec', 'commonName']], suffixes=('_rand',None), how='left', on='commonName')
print(dets[np.isnan(dets['ra'])].dropna())
dets.ra.fillna(dets.ra_rand, inplace=True)
del dets['ra_rand']
dets.dec.fillna(dets.dec_rand, inplace=True)
del dets['dec_rand']

In [ ]:
# Do we want to cut just O3? WE sould construct a custom timebase where time speeds up between observing runs

# This will isolate O3 alone:
#dets = dets[dets.tstamp >1.5535e9]

## Let's Sonify

In [ ]:
# plot some data...
plt.scatter(dets['tstamp'],dets['hrelnorm']**0.5, c=dets['mratbin'], s=dets['mtot']*0.4), 

In [ ]:
# stereo for testing, for production want an ambisonic render (e.g. 'ambiX2' or 'ambix3', higher the better)
chords = [notes]
length = DURATION
system = 'stereo' #'ambiX3'
score =  Score(chords, length)

In [ ]:
sampler = Sampler("./samples/gws")
sampler.preset_details("default")
sampler.modify_preset({"note_length": 20})

In [ ]:
# random directions for now
# phi = 360 * np.random.random(len(dets['mrat']))
# theta = np.arccos(1 - 2 * np.random.random(len(dets['mrat']))) * 180/np.pi

In [ ]:
# Set with globals, but could set here
# PLO = 0

# sonify - remember we sonify backwards in time to align merger times better with chirp!
data = {'azimuth': dets['ra'].values*180/np.pi,
        'polar': dets['dec'].values*180/np.pi,
        'time': 5e9-dets['tstamp'].values,
        'pitch': dets['mratbin'].values,
        'volume': dets['hrelnorm'].values,
        'pitch_shift': dets['mtot'].values
       }


mapvals =  {'azimuth': lambda x : (90-x)%360,
            'polar': lambda x : 90-x,
            'time': lambda x : x,
            'pitch' : lambda x: x,
            'volume' : lambda x : x**0.35,
            'pitch_shift': lambda x : x
           }

maplims =  {'azimuth': (0, 360),
            'polar': (0, 180),
            'time': ('0', '100.01'),
            'pitch' : ('0', '100'),
            'volume' : ('0', '100'),
            'pitch_shift': ('0', '100')
           }

events = Events(data.keys())
events.fromdict(data)
events.apply_mapping_functions(mapvals, maplims, param_lims={'pitch_shift': (PLO, np.log2(dets['mrat'].max()/dets['mrat'].min())*12+PLO)})

In [ ]:
soni = Sonification(score, events, sampler, system)
soni.render()

In [ ]:
# Reverse to get sonification!! (todo - add feature to strauss)
for c in range(len(soni.out_channels)):
    soni.out_channels[str(c)].values = soni.out_channels[str(c)].values[::-1]

In [ ]:
soni.notebook_display()
soni.save(f'gw_sky_{system}.wav')

## Video Making

### Timestamp Layer frames

In [ ]:
Path("text_frames2").mkdir(parents=True, exist_ok=True)
tstamps = -np.linspace(-dets['tstamp'].min()*1.0001, -dets['tstamp'].max(), FPS*DURATION)
flg = 0

for ts in tstamps:
    t = dt.fromtimestamp(ts).strftime('%d / %m / %Y')
    fig=plt.figure(figsize=(5.,1.2))
    ax=fig.add_subplot(1,1,1)
    ax.axis('off')
    plt.text(0.5, 0.5, t,horizontalalignment='center',verticalalignment='center',transform = ax.transAxes, fontsize=52,
            path_effects=[patheffects.withStroke(linewidth=6, foreground='black', capstyle="round")],
            color='w')
    plt.xlim(0,1)
    plt.ylim(0,1)
    plt.savefig(f'text_frames2/tstamp_{flg:05d}.png',dpi=75, transparent=1)
    flg += 1
    plt.close()

### Animation?

🚧🚧 _Work in Progress..._ 🚧🚧

In [ ]:
chan = channels.audio_channels('ambiX9')

Nx = 2560//2
Ny = 1440//2

phis = np.column_stack([np.linspace(0, 2*np.pi, Nx)]*Ny)
thets = np.row_stack([np.linspace(0, np.pi, Ny)]*Nx)

harmlist = []

for i in range(len(chan.mics)):
    harmlist.append(chan.mics[i].antenna(phis,thets))

In [ ]:
total = np.zeros((Nx,Ny))

#framestack = np.dstack([np.dstack(harmlist)]*dets,ra.size)
#print(framestack.size)

tau = 8

kern = lambda framedx, tauframe: (framedx)*np.exp(-(framedx/(tauframe)))


for j in range(len(events.mapping['phi']))[:1]:
    for i in range(len(chan.mics)):
        norm = events.mapping['volume'][j]*chan.mics[i].antenna(events.mapping['phi'][j]*2*np.pi, events.mapping['theta'][j]*np.pi).sum()
        total += norm*harmlist[i]
    
plt.imshow(total.T**5, cmap='cubehelix',extent=(0,2*np.pi,-0.5*np.pi,0.5*np.pi))
plt.scatter(dets['ra'].values, dets['dec'].values, s=dets['hrelnorm'].values*100,c='0.8',edgecolor='k',lw=1)
plt.xlim(0,2*np.pi)
plt.ylim(-0.5*np.pi,0.5*np.pi)

In [ ]:
events.mapping['time']

In [ ]:
from scipy.stats import skewnorm

In [ ]:
ts = np.linspace(-60,60,800)


In [ ]:
print(framestack.sum(-1).sum(-1))

In [ ]:
dets.keys()

In [ ]:
import strauss

In [ ]:
strauss.__file__